In [ ]:
!pip install mediapipe

# Práctica 2: Sensado y análisis de video

Para esta práctica se generó un dataset que contenía 150 videos. Estos videos contienen movimientos utilizados en la escala de evaluación Fugl-Meyer para extremidades superiores. Cada uno de los 10 sujetos realizó 5 movimientos en 3 modalidades (*full*, *partial* y *none*, o completa, parcial y nula). El propósito de la práctica es generar un clasificador que pueda correctamente clasificar un video nuevo como un movimiento *full*, *partial* o *none*.

Comenzamos importando las librerías a utilizar.

In [90]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import sys
import copy
import itertools
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)
import glob

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import joblib

from google.colab import files

Importamos la librería que nos permite cargar datos desde Google Drive, donde tenemos guardados nuestros videos.

In [70]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Establecemos las localizaciones de los videos. Para motivos de entrenamiento se han separado en 5 carpetas según el movimiento, cada una con sus subcarpetas para los movimientos completos, parciales y nulos.

In [71]:
path1 = "/content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento1/"
path2 = "/content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento2/"
path3 = "/content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento3/"
path4 = "/content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento4/"
path5 = "/content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento5/"

In [72]:
class_names = ["Full", "Partial", "None"]

In [73]:
base_paths = [
    path1, path2, path3, path4, path5
]

Se utiliza MediaPipe para extraer los *landmarks*.

In [74]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5)

In [75]:
def extract_pose_landmarks(video_path):
    cap = cv2.VideoCapture(video_path)
    landmarks_list = []

    # Se verifica que el video cargó correctamente
    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return np.array([])

    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Se convierten los frames a RGB
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Se procesan los frames con MediaPipe Pose
        results = pose.process(rgb_frame)

        if results.pose_landmarks:
            # Extrae los landmarks de pose
            landmarks = np.array([[lm.x, lm.y, lm.z] for lm in results.pose_landmarks.landmark]).flatten()
            landmarks_list.append(landmarks)
        else:
            print(f"Warning: No landmarks detected in frame {frame_count} of {video_path}")

    cap.release()
    return np.array(landmarks_list)

La siguiente función se encargará de cargar el dataset para cada una de nuestras carpetas (cada movimiento tendrá su dataset).

In [85]:
def load_dataset_for_path(base_path, class_names):
    X = []
    y = []

    for class_idx, class_name in enumerate(class_names):
        class_folder = os.path.join(base_path, class_name)
        if os.path.exists(class_folder):
            video_files = [f for f in os.listdir(class_folder) if f.lower().endswith(('.mov', '.mp4', '.avi'))]
            for video_file in video_files:
                video_path = os.path.join(class_folder, video_file)
                landmarks = extract_pose_landmarks(video_path)

                # Verifica que los landmarks fueron extraídos
                if landmarks.size > 0:
                    # Calculate a single feature vector for the video (e.g., mean)
                    video_features = np.mean(landmarks, axis=0)  # Average landmarks across all frames

                    X.append(video_features)
                    y.append(class_idx)
                else:
                    print(f"Warning: No landmarks extracted from {video_file}. Skipping...")
        else:
            print(f"Warning: Subfolder '{class_name}' not found in '{base_path}'. Skipping...")

    # Convert to NumPy arrays
    X = np.array(X)
    y = np.array(y)

    return X, y

Se utilizaron RandomForest, KNN, DecisionTree, Regresión Logística y Naive Bayes como clasificadores para entrenar nuestros modelos. Se entrenó un modelo con cada uno de los movimientos. Luego se escogió según sus puntajes el mejor de los modelos, con el cual se haría la prueba externa.

In [88]:
classifiers = [
    GaussianNB(),
    RandomForestClassifier(random_state=42),
    KNeighborsClassifier(),
    DecisionTreeClassifier(random_state=42),
    LogisticRegression(random_state=42, max_iter=1000)
]

for i, base_path in enumerate(base_paths):
    print(f"Training classifier for {base_path}...")

    # Se carga el dataset para la localización actual
    X, y = load_dataset_for_path(base_path, class_names)

    # Selecciona el clasificador para la localización actual
    if i < len(classifiers):  # Verifica que haya un clasificador definido para esta localización
        model = classifiers[i]
    else:
        # RandomForest por defecto si no se especificó un clasificador en concreto
        model = RandomForestClassifier(random_state=42)

    # Se segmentan los datos en datos de entrenamiento y de prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Remove to_categorical for GaussianNB and LogisticRegression
    if isinstance(model, (GaussianNB, LogisticRegression)):
        # GaussianNB and LogisticRegression expect a 1D array for y
        pass  # No need to change y_train and y_test
    else:
        # Other models might work with one-hot encoding
        y_train = to_categorical(y_train)
        y_test = to_categorical(y_test)

    # Entrena el modelo
    model.fit(X_train, y_train)

    # Evalúa el modelo
    y_pred = model.predict(X_test)

    #If the model is GaussianNB or LogisticRegression then reverse the one-hot encoding to calculate the accuracy
    if isinstance(model, (GaussianNB, LogisticRegression)):
        accuracy = accuracy_score(y_test, y_pred)
    else:
        accuracy = accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

    print(f"Test Accuracy for {base_path}: {accuracy * 100:.2f}%")

    # Guarda el modelo
    model_save_path = os.path.join(base_path, f"video_classifier_model_{i+1}.pkl")
    joblib.dump(model, model_save_path)
    print(f"Model saved to {model_save_path}\n")

Training classifier for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento1/...
Test Accuracy for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento1/: 66.67%
Model saved to /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento1/video_classifier_model_1.pkl

Training classifier for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento2/...
Test Accuracy for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento2/: 66.67%
Model saved to /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento2/video_classifier_model_2.pkl

Training classifier for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento3/...
Test Accuracy for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento3/: 16.67%
Model saved to /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento3/video_classifier_model_3.pkl

Training classifier for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento4/...
Test Accuracy for /content/drive/MyDrive/CDSI/Practica02/Videos/Movimiento4/: 66.67%
Model s

# Prueba con video nuevo

In [92]:
uploaded = files.upload()
video_file_name = list(uploaded.keys())[0]  # Get the name of the uploaded file

# Extract features from the uploaded video
landmarks = extract_pose_landmarks(video_file_name)

# Check if landmarks were extracted
if landmarks.size == 0:
    print("No landmarks detected in the video.")
else:
    # Load the trained model
    # Load the trained model using joblib.load()
    model_path = "/content/drive/My Drive/CDSI/Practica02/Videos/Movimiento1/video_classifier_model_1.pkl"
    model = joblib.load(model_path)


    # Normalize the landmarks (if required)
    # Example: If you used StandardScaler during training, load and apply it here
    # from sklearn.preprocessing import StandardScaler
    # scaler = StandardScaler()
    # landmarks = scaler.transform(landmarks)

    # Predict the class
    predictions = model.predict(landmarks)
    predicted_class_idx = np.argmax(np.mean(predictions, axis=0))  # Average predictions over all frames
    class_names = ["full", "partial", "none"]
    predicted_class = class_names[predicted_class_idx]

    # Display the result
    print(f"The uploaded video is classified as: {predicted_class}")

Saving WhatsApp Video 2025-02-12 at 9.57.26 AM.mp4 to WhatsApp Video 2025-02-12 at 9.57.26 AM (1).mp4
The uploaded video is classified as: full
